In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

from scipy.spatial.distance import euclidean, pdist, squareform

import warnings; warnings.simplefilter('ignore')

In [36]:
md_top=pd.read_csv('./md_top.csv')
titles=pd.read_csv('./title.csv')
indices=pd.read_csv('./indices.csv')
indice1=pd.Series(indices.index,index=indices['title'])


In [37]:
tf2=TfidfVectorizer(analyzer='word',min_df=0, stop_words='english', ngram_range=(1,1))

In [38]:
tfidfmat2=tf2.fit_transform(md_top['soup'].values.astype('U'))

In [39]:
tfidfmat2.shape

(22286, 52239)

In [40]:
sim_mat=linear_kernel(tfidfmat2,tfidfmat2)

In [41]:
def movie_recf(title):
    idx=indice1[title.lower()]
    if isinstance(idx, pd.Series):
        idx=idx.values[0]
    args=np.argsort(sim_mat[idx])
    #print(sim_mat[idx])
    args=args[-31:]
    args=reversed(args)
    inds=[i for i in args]
    movies= md_top.loc[inds][['title','Director_','cast','vote_average','wr']]
    movies =movies.sort_values('wr',ascending=False)
    return movies

<h2><b>Some Examples</b></h2>

In [42]:
movie_recf('the dark knight')

,title,Director_,cast,vote_average,wr
7528,The Dark Knight,Christopher Nolan,"['christianbale', 'michaelcaine', 'heathledger']",8.3,8.282195
9124,Inception,Christopher Nolan,"['leonardodicaprio', 'josephgordon-levitt', 'e...",8.1,8.085625
12676,Interstellar,Christopher Nolan,"['matthewmcconaughey', 'jessicachastain', 'ann...",8.1,8.081941
2463,Memento,Christopher Nolan,"['guypearce', 'carrie-annemoss', 'joepantoliano']",8.1,8.052116
6845,The Prestige,Christopher Nolan,"['hughjackman', 'christianbale', 'michaelcaine']",8.0,7.957468
10595,The Dark Knight Rises,Christopher Nolan,"['christianbale', 'michaelcaine', 'garyoldman']",7.6,7.582610
11522,"Batman: The Dark Knight Returns, Part 2",Jay Oliva,"['peterweller', 'arielwinter', 'davidselby']",7.9,7.531679
6091,Batman Begins,Christopher Nolan,"['christianbale', 'michaelcaine', 'liamneeson']",7.5,7.479678
21957,Dunkirk,Christopher Nolan,"['fionnwhitehead', 'tomglynn-carney', 'jacklow...",7.5,7.444772
9139,Batman: Under the Red Hood,Brandon Vietti,"['brucegreenwood', 'jensenackles', 'neilpatric...",7.6,7.299617


In [43]:
movie_recf('the avengers')

,title,Director_,cast,vote_average,wr
13030,Guardians of the Galaxy,James Gunn,"['chrispratt', 'zoesaldana', 'davebautista']",7.9,7.881467
12760,Captain America: The Winter Soldier,Anthony Russo,"['chrisevans', 'samuell.jackson', 'scarlettjoh...",7.6,7.572747
10378,The Avengers,Joss Whedon,"['robertdowneyjr.', 'chrisevans', 'markruffalo']",7.4,7.387907
14142,Deadpool,Tim Miller,"['ryanreynolds', 'morenabaccarin', 'edskrein']",7.4,7.387324
7596,Iron Man,Jon Favreau,"['robertdowneyjr.', 'terrencehoward', 'jeffbri...",7.4,7.383825
6150,Serenity,Joss Whedon,"['nathanfillion', 'ginatorres', 'alantudyk']",7.4,7.293275
14139,Avengers: Age of Ultron,Joss Whedon,"['robertdowneyjr.', 'chrishemsworth', 'markruf...",7.3,7.280271
8116,Dr. Horrible's Sing-Along Blog,Joss Whedon,"['neilpatrickharris', 'nathanfillion', 'felici...",7.8,7.237399
14144,Captain America: Civil War,Anthony Russo,"['chrisevans', 'robertdowneyjr.', 'scarlettjoh...",7.1,7.083894
14145,Doctor Strange,Scott Derrickson,"['benedictcumberbatch', 'chiwetelejiofor', 'ra...",7.1,7.079620


In [46]:
movie_recf('the wolf of wall street')

,title,Director_,cast,vote_average,wr
72,Taxi Driver,Martin Scorsese,"['robertdeniro', 'cybillshepherd', 'harveykeit...",8.1,8.025016
12351,The Wolf of Wall Street,Martin Scorsese,"['leonardodicaprio', 'jonahhill', 'margotrobbie']",7.9,7.872685
6816,The Departed,Martin Scorsese,"['leonardodicaprio', 'mattdamon', 'jacknichols...",7.9,7.858760
8809,Shutter Island,Martin Scorsese,"['leonardodicaprio', 'markruffalo', 'benkingsl...",7.8,7.773060
10,Casino,Martin Scorsese,"['robertdeniro', 'sharonstone', 'joepesci']",7.8,7.674451
726,Raging Bull,Martin Scorsese,"['robertdeniro', 'joepesci', 'cathymoriarty']",7.7,7.537422
3428,The King of Comedy,Martin Scorsese,"['robertdeniro', 'jerrylewis', 'diahnneabbott']",7.6,7.181168
3404,Gangs of New York,Martin Scorsese,"['leonardodicaprio', 'danielday-lewis', 'camer...",7.1,7.040612
5737,The Aviator,Martin Scorsese,"['leonardodicaprio', 'cateblanchett', 'katebec...",7.0,6.929535
4492,Mean Streets,Martin Scorsese,"['harveykeitel', 'robertdeniro', 'davidproval']",7.2,6.905880


In [49]:
movie_recf('how to train your dragon')

,title,Director_,cast,vote_average,wr
2106,Gladiator,Ridley Scott,"['russellcrowe', 'joaquinphoenix', 'connieniel...",7.9,7.866872
10382,Warrior,Gavin O'Connor,"['tomhardy', 'jennifermorrison', 'nicknolte']",7.7,7.576210
12960,How to Train Your Dragon 2,Dean DeBlois,"['jaybaruchel', 'gerardbutler', 'kristenwiig']",7.6,7.549921
1078,Mulan,Tony Bancroft,"['ming-nawen', 'leasalonga', 'eddiemurphy']",7.6,7.525146
8886,How to Train Your Dragon,Chris Sanders,"['jaybaruchel', 'gerardbutler', 'craigferguson']",7.5,7.464938
628,Platoon,Oliver Stone,"['tomberenger', 'charliesheen', 'willemdafoe']",7.5,7.382923
13735,The Hobbit: The Battle of the Five Armies,Peter Jackson,"['martinfreeman', 'ianmckellen', 'richardarmit...",7.1,7.075532
3126,Lilo & Stitch,Dean DeBlois,"['chrissanders', 'daveighchase', 'tiacarrere']",7.1,7.016375
11678,The Croods,Chris Sanders,"['nicolascage', 'emmastone', 'ryanreynolds']",6.8,6.761682
17232,Dragons: Gift of the Night Fury,Tom Owens,"['jaybaruchel', 'gerardbutler', 'craigferguson']",6.9,6.394678
